<a href="https://colab.research.google.com/github/kushv16/Audio-Sentiment-Analysis/blob/master/EMOV_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install noisereduce

import pandas as pd
import numpy as np
import random
from glob import glob
import os
import librosa
import noisereduce as nr
from scipy.io import wavfile
import warnings
warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Import the dataset and verify that the upload is successful
df = pd.read_csv('/content/drive/MyDrive/LY Project/Dataframes/Dataset/emov-db-df.csv', index_col=0)
print('The dataset has {} audio files'.format(df.shape[0]))
df.sample()

The dataset has 6893 audio files


,path,filename,dataset,duration,sample_rate,gender,age,emotion
1275,/kaggle/input/emovdb-sorted/bea/Neutral/neutra...,neutral_337-364_0350,EmoV_DB,4.924,44100,female,30,neutral


In [4]:
paths = list(df['path'])

updated_paths = []
updated_filenames = []

for path in paths:
  path_components = path.split('/')
  updated_path_components = path_components[4:]
  path_string = "/"
  a = "/content/drive/MyDrive/Emov-db/" + path_string.join(updated_path_components)
  updated_paths.append(a)
  
  components = a.split('/')
  updated_filename = components[5] + "_" + components[-1]
  updated_filenames.append(updated_filename)
  
  

In [5]:
df['path'] = updated_paths
df['filename'] = updated_filenames
df.head(5)

,path,filename,dataset,duration,sample_rate,gender,age,emotion
0,/content/drive/MyDrive/Emov-db/bea/Amused/amus...,bea_amused_1-15_0001.wav,EmoV_DB,4.385,44100,female,30,amused
1,/content/drive/MyDrive/Emov-db/bea/Amused/amus...,bea_amused_1-15_0002.wav,EmoV_DB,4.944,44100,female,30,amused
2,/content/drive/MyDrive/Emov-db/bea/Amused/amus...,bea_amused_1-15_0003.wav,EmoV_DB,6.399,44100,female,30,amused
3,/content/drive/MyDrive/Emov-db/bea/Amused/amus...,bea_amused_1-15_0004.wav,EmoV_DB,4.058,44100,female,30,amused
4,/content/drive/MyDrive/Emov-db/bea/Amused/amus...,bea_amused_1-15_0005.wav,EmoV_DB,3.358,44100,female,30,amused


In [6]:
t = os.listdir("/content/drive/MyDrive/LY Project/Cleaned Datasets/cleaned_samples_emodb")
len(t)

6893

In [ ]:
cd "/content/drive/MyDrive/LY Project/Cleaned Datasets/cleaned_samples_emodb"

/content/drive/.shortcut-targets-by-id/1ty6Q2chm7khBo3iSNsdx5NVqeOpCUmjM/LY Project/Cleaned Datasets/cleaned_samples_emodb


In [7]:
!pip install tqdm

from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
# saving the new cleaned files path in a variable
cleaned_files = sorted(filter(os.path.isfile, glob('/content/drive/MyDrive/LY Project/Cleaned Datasets/cleaned_samples_emodb/*.wav')),key=os.path.getmtime)

In [9]:
cleaned_files[:-100]

['/content/drive/MyDrive/LY Project/Cleaned Datasets/cleaned_samples_emodb/sam_disgust_477-504_0490_cleaned.wav',
 '/content/drive/MyDrive/LY Project/Cleaned Datasets/cleaned_samples_emodb/sam_disgust_477-504_0491_cleaned.wav',
 '/content/drive/MyDrive/LY Project/Cleaned Datasets/cleaned_samples_emodb/sam_disgust_477-504_0492_cleaned.wav',
 '/content/drive/MyDrive/LY Project/Cleaned Datasets/cleaned_samples_emodb/sam_disgust_477-504_0494_cleaned.wav',
 '/content/drive/MyDrive/LY Project/Cleaned Datasets/cleaned_samples_emodb/sam_disgust_477-504_0496_cleaned.wav',
 '/content/drive/MyDrive/LY Project/Cleaned Datasets/cleaned_samples_emodb/sam_disgust_477-504_0497_cleaned.wav',
 '/content/drive/MyDrive/LY Project/Cleaned Datasets/cleaned_samples_emodb/sam_disgust_477-504_0498_cleaned.wav',
 '/content/drive/MyDrive/LY Project/Cleaned Datasets/cleaned_samples_emodb/sam_disgust_477-504_0499_cleaned.wav',
 '/content/drive/MyDrive/LY Project/Cleaned Datasets/cleaned_samples_emodb/sam_disgust_4

In [ ]:
f0_mean, f0_median, f0_std, f0_0, f0_25, f0_75, f0_100, filename = [], [], [], [], [], [], [], []

# Finction to extract f0 values
def get_f0(file):
    # load audio file:
    y, sr = librosa.load(file, sr=16000)
    filename.append(file)
    # Extract fundamental frequency using a probabilistic approach
    f_zero, _, _ = librosa.pyin(y, sr=sr, fmin=50, fmax=1500, frame_length=1024)

    f0_mean.append(np.nanmean(f_zero))             # f0 mean
    f0_median.append(np.nanmedian(f_zero))         # f0 median
    f0_std.append(np.nanstd(f_zero))               # f0 standar deviation
    f0_0.append(np.nanpercentile(f_zero, 0))       # f0 min
    f0_25.append(np.nanpercentile(f_zero, 25))     # f0 25%-percentile value
    f0_75.append(np.nanpercentile(f_zero, 75))     # f0 75%-percentile value
    f0_100.append(np.nanpercentile(f_zero, 100))   # f0 max

In [ ]:
%%time

# Apply function to cleaned files
for file in tqdm(cleaned_files):
    get_f0(file)

100%|██████████| 6893/6893 [4:51:16<00:00,  2.54s/it]

CPU times: user 4h 1min 3s, sys: 3min 58s, total: 4h 5min 2s
Wall time: 4h 51min 16s


In [ ]:
df_f0 = pd.DataFrame()
df_f0['f0_mean'] = f0_mean
df_f0['f0_median'] = f0_median
df_f0['f0_std'] = f0_std
df_f0['f0_0'] = f0_0
df_f0['f0_25'] = f0_25
df_f0['f0_75'] = f0_75
df_f0['f0_100'] = f0_100
df_f0['filename'] = filename
df_f0.sample()

,f0_mean,f0_median,f0_std,f0_0,f0_25,f0_75,f0_100,filename
3940,150.522382,147.256706,25.543264,97.153194,129.683955,169.156358,203.495938,/content/drive/MyDrive/LY Project/Cleaned Data...


In [ ]:
df_f0.to_csv("emodb_fund.csv")

In [ ]:
pwd

'/content/drive/.shortcut-targets-by-id/1ty6Q2chm7khBo3iSNsdx5NVqeOpCUmjM/LY Project/Cleaned Datasets/cleaned_samples_emodb'

### <a id='2_b2'>Other variables</a>

In [10]:
%%time

# declare variables where to store the extracted feaures
zcr_mean, zcr_var, spectral_centroid_mean, spectral_centroid_var, rms_mean, rms_var, chroma_stft_mean, chroma_stft_var = [], [], [], [], [], [], [], []
rolloff_mean, rolloff_var, spectral_bandwidth_mean, spectral_bandwidth_var, harmony_mean, harmony_var, chroma_cqt_mean, chroma_cqt_var = [], [], [], [], [], [], [], []
spectral_contrast_mean, spectral_contrast_var, spectral_flatness_mean, spectral_flatness_var, chroma_cens_mean, chroma_cens_var = [], [], [], [], [], []
filename = []
# extract features
for file in tqdm(cleaned_files):
    # load audio file:
    y, sr = librosa.load(file, sr=16000)
    filename.append(file)
    # Zero crossing rate
    zcr_mean.append(np.mean(librosa.feature.zero_crossing_rate(y)))
    zcr_var.append(np.var(librosa.feature.zero_crossing_rate(y)))
    
    # Spectral centroid
    spectral_centroid_mean.append(np.mean(librosa.feature.spectral_centroid(y)))
    spectral_centroid_var.append(np.var(librosa.feature.spectral_centroid(y)))
    
    # Spectral contrast
    spectral_contrast_mean.append(np.mean(librosa.feature.spectral_contrast(y)))
    spectral_contrast_var.append(np.var(librosa.feature.spectral_contrast(y)))
    
    # Spectral flatness
    spectral_flatness_mean.append(np.mean(librosa.feature.spectral_flatness(y)))
    spectral_flatness_var.append(np.var(librosa.feature.spectral_flatness(y)))
    
    # Rms mean and variance
    rms_mean.append(np.mean(librosa.feature.rms(y=y)))               
    rms_var.append(np.var(librosa.feature.rms(y=y)))
    
    # Chroma mean and variance
    chroma_stft_mean.append(np.mean(librosa.feature.chroma_stft(y, sr)))                   
    chroma_stft_var.append(np.var(librosa.feature.chroma_stft(y, sr)))
    
    #Chroma cqt mean and variance
    chroma_cqt_mean.append(np.mean(librosa.feature.chroma_cqt(y, sr)))                   
    chroma_cqt_var.append(np.var(librosa.feature.chroma_cqt(y, sr)))
    
    #Chroma cens mean and variance
    chroma_cens_mean.append(np.mean(librosa.feature.chroma_cens(y, sr)))                   
    chroma_cens_var.append(np.var(librosa.feature.chroma_cens(y, sr)))
    
    # Rolloff mean and variance
    rolloff_mean.append(np.mean(librosa.feature.spectral_rolloff(y,sr)))
    rolloff_var.append(np.var(librosa.feature.spectral_rolloff(y,sr)))
    
    # Spectral bandwidth mean and variance
    spectral_bandwidth_mean.append(np.mean(librosa.feature.spectral_bandwidth(y,sr)))
    spectral_bandwidth_var.append(np.var(librosa.feature.spectral_bandwidth(y,sr)))
    
    # Harmony mean and variance
    harmony_mean.append(np.mean(librosa.effects.harmonic(y)))
    harmony_var.append(np.var(librosa.effects.harmonic(y)))

100%|██████████| 6893/6893 [2:55:47<00:00,  1.53s/it]

CPU times: user 2h 26min 27s, sys: 40min 56s, total: 3h 7min 24s
Wall time: 2h 55min 47s


In [11]:
# add new features to a new dataframe
df_features = pd.DataFrame()
df_features['zcr_mean'] = zcr_mean
df_features['zcr_var'] = zcr_var
df_features['spectral_centroid_mean'] = spectral_centroid_mean
df_features['spectral_centroid_var'] = spectral_centroid_var
df_features['spectral_contrast_mean'] = spectral_contrast_mean
df_features['spectral_contrast_var'] = spectral_contrast_var
df_features['spectral_flatness_mean'] = spectral_flatness_mean
df_features['spectral_flatness_var'] = spectral_flatness_var
df_features['rms_mean'] = rms_mean
df_features['rms_var'] = rms_var
df_features['chroma_stft_mean'] = chroma_stft_mean
df_features['chroma_stft_var'] = chroma_stft_var
df_features['chroma_cqt_mean'] = chroma_cqt_mean
df_features['chroma_cqt_var'] = chroma_cqt_var
df_features['chroma_cens_mean'] = chroma_cens_mean
df_features['chroma_cens_var'] = chroma_cens_var
df_features['spectral_bandwidth_mean'] = spectral_bandwidth_mean
df_features['spectral_bandwidth_var'] = spectral_bandwidth_var
df_features['rolloff_mean'] = rolloff_mean
df_features['rolloff_var'] = rolloff_var
df_features['harmony_mean'] = harmony_mean
df_features['harmony_var'] = harmony_var
df_features['filename'] = filename
# Check a random row
df_features.sample()

,zcr_mean,zcr_var,spectral_centroid_mean,spectral_centroid_var,spectral_contrast_mean,spectral_contrast_var,spectral_flatness_mean,spectral_flatness_var,rms_mean,rms_var,...,chroma_cqt_var,chroma_cens_mean,chroma_cens_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,harmony_mean,harmony_var,filename
2158,0.108238,0.009912,1979.879958,1.658756e+06,19.267003,30.971047,0.02036,0.001949,0.095014,0.006113,...,0.081115,0.250604,0.020531,1338.298238,184658.02267,2695.200893,2.363633e+06,-0.000001,0.006828,/content/drive/MyDrive/LY Project/Cleaned Data...


In [19]:
df_features.shape

(6893, 23)

In [14]:
df_features.to_csv("/content/drive/MyDrive/LY Project/Dataframes/Features/EMO/emo_misc_df.csv")

### <a id='2_b3'>MFCCs</a>

In [17]:
# function to extract mean and variance of Mel-Frequency Cepstrum Components (MFCCs)
def extract_mfcc(file):
    
    mfcc_mean, mfcc_var = [], []
    
    # load audio file:
    y, sr = librosa.load(file, sr=16000)
    
    # get 20 mean values
    mfcc_mean.append(np.mean(librosa.feature.mfcc(y=y, sr=sr, fmin=50, n_mfcc=30).T, axis=0))
    
    # get 20 variance values
    mfcc_var.append(np.var(librosa.feature.mfcc(y=y, sr=sr, fmin=50, n_mfcc=30).T, axis=0))
    
    return np.hstack((mfcc_mean, mfcc_var))[0]

# apply function above to all files and store results in a list
extracted_mfcc = []
filename = []
for file in cleaned_files:
    filename.append(file)
    extracted_mfcc.append(extract_mfcc(file))

# create column names for a dataframe
name_mfcc_mean, name_mfcc_var = [], []

for i in range(0, 30):
    name_mfcc_mean.append('mfcc'+str(i+1)+'_mean')   # mfcc1_mean, mfcc2_mean, ...
    name_mfcc_var.append('mfcc'+str(i+1)+'_var')     # mfcc1_var, mfcc2_var, ...

name_mfcc = name_mfcc_mean + name_mfcc_var           # concatenate mfcc mean and variance names

# finally put all mfcc values in a dataframe
df_mfcc = pd.DataFrame(extracted_mfcc, columns = name_mfcc)
df_mfcc['filename'] = filename
df_mfcc.sample()      # print one sample

,mfcc1_mean,mfcc2_mean,mfcc3_mean,mfcc4_mean,mfcc5_mean,mfcc6_mean,mfcc7_mean,mfcc8_mean,mfcc9_mean,mfcc10_mean,...,mfcc22_var,mfcc23_var,mfcc24_var,mfcc25_var,mfcc26_var,mfcc27_var,mfcc28_var,mfcc29_var,mfcc30_var,filename
3944,-203.621399,108.6437,-4.811107,15.246647,17.588278,2.470445,-0.576775,-14.980326,-12.328325,-4.069848,...,46.665253,46.787971,33.73592,33.577171,28.909891,34.809139,28.202105,38.821236,35.439926,/content/drive/MyDrive/LY Project/Cleaned Data...


In [20]:
df_mfcc.to_csv("/content/drive/MyDrive/LY Project/Dataframes/Features/emo_mfcc_df.csv")